## 1. Apresentação  

### Objetivo

Seu desafio é desenvolver um modelo de sistema de recomendação e realizar o deploy dele utilizando as técnicas aprendidas no curso. Nesse cenário, surge o desafio de fornecer recomendações personalizadas para cada usuário com base nos dados de notícias do DPlay, predizendo qual será a próxima notícia que ele vai ler.

Técnica Aprendizagem supervisionada, mais precisamente a técnicas de recomendação.

### Conjunto de Treino 

O conjunto de treino está disponibilizado em diferentes partes, cada uma contendo informação complementar.
Os arquivos treino_parte_X.csv, em que X é um valor de 1 até 6, consistem das colunas:
    
    
    1. userId: id do usuário.
    2. userType: usuário logado ou anônimo.
    3. HistorySize: quantidade de notícias lidas pelo usuário.
    4. history: lista de notícias visitadas pelo usuário.
    5. TimestampHistory: momento em que o usuário visitou a página.
    6. timeOnPageHistory: quantidade de ms em que o usuário ficou na página.
    7. numberOfClicksHistory: quantidade de clicks na matéria.
    8. scrollPercentageHistory: quanto o usuário visualizou da matéria.
    9. pageVisitsCountHistory: quantidade de vezes que o usuário visitou a matéria.


Além desses arquivos, a pasta de treino contém uma subpasta denominada de itens. Ela contém a seguinte informação:

    1. Page: id da matéria. Esse é o mesmo id que aparece na coluna history de antes.
    2. Url: url da matéria.
    3. Issued: data em que a matéria foi criada.
    4. Modified: última data em que a matéria foi modificada.
    5. Title: título da matéria.
    6. Body: corpo da matéria.Caption: subtítulo da matéria.

Este conjunto de treino consiste em dados de usuários reais da DPlay. Eles forem coletados até uma data limite T (a maior data em todo o conjunto TimestampHistory).


### Conjunto de Validação

Capturando informações de um período posterior ao treino, ou seja, não existe sobreposição temporal com o treino, o conjunto de validação consiste em:

    1. userId: id do usuário.
    2. userType: usuário logado ou anônimo.
    3. history: lista de páginas que devem ser recomendadas ao usuário.



## 2. Carregamento dos Dados

In [1]:
import pandas as pd
import numpy as np

from nltk.corpus import stopwords
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import re
import random 


from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
import string
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
import plotly
import plotly.offline as py
import plotly.graph_objs as go # criará de fato os gráficos
from plotly.offline import plot, iplot
import cufflinks as cf # para conectar o plotly ao pandas
cf.go_offline()
plotly.offline.init_notebook_mode(connected = True)
pd.options.display.max_columns = 30
cf.set_config_file(world_readable=True, theme='solar')
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

import plotly.io as pio
pio.renderers


Renderers configuration
-----------------------
    Default renderer: 'plotly_mimetype+notebook_connected'
    Available renderers:
        ['plotly_mimetype', 'jupyterlab', 'nteract', 'vscode',
         'notebook', 'notebook_connected', 'kaggle', 'azure', 'colab',
         'cocalc', 'databricks', 'json', 'png', 'jpeg', 'jpg', 'svg',
         'pdf', 'browser', 'firefox', 'chrome', 'chromium', 'iframe',
         'iframe_connected', 'sphinx_gallery', 'sphinx_gallery_png']

In [3]:
pio.renderers.default = 'iframe'
print("Done")

Done


### Adicionando StopWords PT-BR

In [4]:
stopwords_ptbr = [
    "a", "à", "agora", "ainda", "alguém", "algum", "alguma", "algumas", "alguns", "ampla", "amplas", "amplo", "amplos",
    "ante", "antes", "ao", "aos", "após", "aquela", "aquelas", "aquele", "aqueles", "aquilo", "as", "às", "até", "atrás", "bem", "boa", "boas", "bom", "bons",
    "cada", "caminho", "catorze", "cedo", "cento", "certamente", "certeza", "cima", "cinco", "coisa", "coisas", "com", "como", "conselho",
    "contudo", "da", "daquele", "daqueles", "dar", "das", "de", "debaixo", "dela", "delas", "dele", "deles", "demais", "dentro", "depois", "desde",
    "dessa", "dessas", "desse", "desses", "desta", "destas", "deste", "destes", "deve", "devem", "deverá", "dez", "dezoito", "dia", "diante", "do",
    "dois","dos", "doze", "duas", "durante", "e", "ela", "elas", "ele", "eles", "em", "embora", "entre", "então", "era", "essa", "essas", "esse", "esses",
    "esta", "está", "estamos", "estão", "estar", "estas", "estava", "estavam", "este", "estes", "esteve", "estive", "estivemos", "estiveram",
    "eu", "exemplo", "fará", "fazer", "fez", "fim", "foi", "fora", "foram", "for", "fosse", "fossem", "fui", "geral", "grande", "grandes", "grupo",
    "há", "haja", "houve", "ia", "início", "ir", "isso", "isto", "já", "lado", "lá", "lhe", "lhes", "logo", "longe", "lugar", "maior", "mais",
    "mal", "mas", "me", "meio", "menor", "menos", "meu", "meus", "mil", "minha", "minhas", "momento", "muito", "muitos", "na", "nas", "nada", "não", "naquela",
    "naquelas", "naquele", "naqueles", "nas", "nem", "nenhum", "nessa", "nessas", "nesse", "nesses", "nesta", "nestas", "neste", "nestes", "ninguém",
    "nas", "no", "nós", "nos", "nossa", "nossas", "nosso", "nossos", "nova", "novas", "nove", "novo", "novos", "num", "numa", "número", "nunca", "o", "onde",
    "ontem", "onze", "os", "ou", "outra", "outras", "outro", "outros", "para", "parece", "parte", "pegar", "pela", "pelas", "pelo", "pelos", "perto",
    "pode", "pude", "põem", "ponto", "pontos", "por", "porque", "porquê", "posição", "possível", "posso", "pouca", "poucas", "pouco", "poucos",
    "primeiro", "próprio", "próxima", "próximas", "próximo", "próximos", "puderam", "quando", "quanto", "quase", "quatro", "que", "quem", "quer",
    "quero", "questão", "quieto", "quinta", "quinto", "quiseram", "quatorze", "quinhentos", "relação", "sabe", "são", "se", "segunda", "segundo",
    "sei", "seis", "sem", "sempre", "ser", "será", "serão", "sete", "seu", "seus", "sexta", "sexto", "sim", "sistema", "sob", "sobre", "sois", "somos",
    "sou", "sua", "suas", "tal", "talvez", "também", "tanta", "tantas", "tanto", "tantos", "te", "tem", "têm", "temos", "tempo", "tenho", "ter",
    "terá", "terão", "terceira", "terceiro", "teu", "teus", "teve", "tinha", "tinham", "toda", "todas", "todo", "todos", "trabalho", "três", "treze",
    "tu", "tua", "tuas", "tudo", "último", "um", "uma", "umas", "uns", "vai", "valor", "veja", "ver", "verdade", "verdadeiro", "vez", "vezes", "vi",
    "vindo", "vinte", "você", "vocês", "vos", "vossa", "vossas", "vosso", "vossos", "zero"
]
print("Done")

Done


## Configurações da AWS - S3

In [5]:
import os
import boto3
import pickle
from io import BytesIO
from kaggle_secrets import UserSecretsClient
from botocore.exceptions import NoCredentialsError

user_secrets = UserSecretsClient()

# Configurações da AWS
AWS_ACCESS_KEY = user_secrets.get_secret("AWS_ACCESS_KEY_ID")
AWS_SECRET_KEY = user_secrets.get_secret("AWS_SECRET_ACCESS_KEY")
AWS_SESSION_TOKEN = user_secrets.get_secret("AWS_SESSION_TOKEN")
AWS_REGION = user_secrets.get_secret("AWS_REGION")
BUCKET_NAME = "datathon-base"

# Criar conexão com o S3
def get_s3_client():
    return boto3.client(
        "s3",
        aws_access_key_id=AWS_ACCESS_KEY,
        aws_secret_access_key=AWS_SECRET_KEY,
        aws_session_token=AWS_SESSION_TOKEN,
        region_name=AWS_REGION,
    )

def upload_to_s3(file_name, bucket_name, object_name=None):
    """
    Faz upload de um arquivo CSV para um bucket S3.

    :param file_name: Caminho do arquivo local a ser enviado
    :param bucket_name: Nome do bucket S3
    :param object_name: Nome do arquivo no S3 (opcional, usa o nome original se None)
    :param aws_access_key: Chave de acesso AWS (opcional se já configurado)
    :param aws_secret_key: Chave secreta AWS (opcional se já configurado)
    """
    try:
        # Criar cliente S3
        s3_client = get_s3_client()
        
        if object_name is None:
            object_name = file_name.split("/")[-1]  # Usa apenas o nome do arquivo se não especificado
        
        s3_client.upload_file(file_name, bucket_name, object_name)
        print(f"Upload de {file_name} para {bucket_name}/{object_name} concluído com sucesso.")
    except NoCredentialsError:
        print("Credenciais não encontradas. Verifique suas configurações de autenticação AWS.")
    except Exception as e:
        print(f"Erro ao fazer upload: {e}")
        
def upload_to_s3(file_path, bucket_name=BUCKET_NAME, object_name=None):
    """Faz upload de um arquivo local para um bucket S3."""
    
    if object_name is None:
        object_name = os.path.basename(file_path)  # Nome padrão do arquivo no S3

    try:
        s3_client = get_s3_client()

        s3_client.upload_file(file_path, bucket_name, object_name)
        print(f"✔ Upload de {file_path} para s3://{bucket_name}/{object_name} concluído.")
        return True

    except FileNotFoundError:
        print("❌ O arquivo não foi encontrado.")
        return False
    except NoCredentialsError:
        print("❌ Credenciais da AWS não encontradas.")
        return False

# Função para salvar o modelo no S3
def save_model_to_s3(model_data, s3_path="models/news_recommendation_model_cold_start.pkl"):
    s3 = get_s3_client()
    
    # Serializar o modelo
    with open("news_recommendation_model_cold_start.pkl", "wb") as f:
        pickle.dump(model_data, f)

    # Fazer upload para o S3
    with open("news_recommendation_model_cold_start.pkl", "rb") as f:
        s3.upload_fileobj(f, BUCKET_NAME, s3_path)

    print(f"Modelo salvo no S3 em: s3://{BUCKET_NAME}/{s3_path}")

# Função para carregar o modelo do S3
def load_model_from_s3(s3_path="models/news_recommendation_model_cold_start.pkl"):
    s3 = get_s3_client()
    
    # Baixar o arquivo do S3
    with open("news_recommendation_model_cold_start.pkl", "wb") as f:
        s3.download_fileobj(BUCKET_NAME, s3_path, f)

    # Carregar o modelo
    with open("news_recommendation_model_cold_start.pkl", "rb") as f:
        model_data = pickle.load(f)

    print("Modelo Cold Start carregado com sucesso!")
    return model_data

def download_dataframe_from_s3(filename):
    """
    Baixa um arquivo Parquet do S3 e carrega como DataFrame.
    
    :param filename: Nome do arquivo no S3 (ex: "dados/meu_arquivo.parquet")
    :return: DataFrame do pandas
    """
    try:
        s3 = get_s3_client()
        response = s3.get_object(Bucket=BUCKET_NAME, Key=filename)
        buffer = BytesIO(response["Body"].read())
        df = pd.read_parquet(buffer, engine="pyarrow")
        print(f"Arquivo {filename} baixado com sucesso!")
        return df
    except Exception as e:
        print("Erro ao baixar arquivo do S3:", str(e))
        return None

def list_s3_files(bucket_name, prefix=''):
    """
    Lista os arquivos de um bucket S3.
    
    :param bucket_name: Nome do bucket
    :param prefix: Prefixo opcional para filtrar os arquivos
    :return: Lista de arquivos no bucket
    """
    s3 = get_s3_client() 
    files = []
    
    try:
        response = s3.list_objects_v2(Bucket=BUCKET_NAME, Prefix=prefix)
        if 'Contents' in response:
            files = [obj['Key'] for obj in response['Contents']]
    except Exception as e:
        print(f"Erro ao listar arquivos do bucket {bucket_name}: {e}")
    
    return files

print("Done")

Done


In [6]:
arquivos = list_s3_files(BUCKET_NAME)
arquivos


['base_original/interacoes/dados_2022_07.csv',
 'base_original/interacoes/dados_2022_08.csv',
 'base_original/noticias/dados_2014_07.csv',
 'base_original/noticias/dados_2015_12.csv',
 'base_original/noticias/dados_2016_10.csv',
 'base_original/noticias/dados_2016_11.csv',
 'base_original/noticias/dados_2016_12.csv',
 'base_original/noticias/dados_2017_01.csv',
 'base_original/noticias/dados_2017_02.csv',
 'base_original/noticias/dados_2017_03.csv',
 'base_original/noticias/dados_2017_04.csv',
 'base_original/noticias/dados_2017_05.csv',
 'base_original/noticias/dados_2017_06.csv',
 'base_original/noticias/dados_2017_07.csv',
 'base_original/noticias/dados_2017_08.csv',
 'base_original/noticias/dados_2017_09.csv',
 'base_original/noticias/dados_2017_10.csv',
 'base_original/noticias/dados_2017_11.csv',
 'base_original/noticias/dados_2017_12.csv',
 'base_original/noticias/dados_2018_01.csv',
 'base_original/noticias/dados_2018_02.csv',
 'base_original/noticias/dados_2018_03.csv',
 'base

In [7]:
class S3FileDownloader:
    def __init__(self, bucket_name, prefix, local_dir='downloads', arq ='', new_arq =''):
        """
        Classe para baixar arquivos de um diretório no S3 e concatená-los em um único DataFrame.
        
        :param bucket_name: Nome do bucket S3
        :param prefix: Prefixo do diretório no S3
        :param local_dir: Diretório local para salvar os arquivos (padrão: 'downloads')
        """
        self.s3_client = get_s3_client()
        self.bucket_name = bucket_name
        self.prefix = prefix
        self.local_dir = local_dir
        self.arq = arq
        self.new_arq = new_arq
        
        if not os.path.exists(local_dir):
            os.makedirs(local_dir)
    
    def list_s3_files(self):
        """Lista os arquivos CSV no bucket S3 dentro do diretório especificado."""
        files = []
        try:
            response = self.s3_client.list_objects_v2(Bucket=self.bucket_name, Prefix=self.prefix)
            if 'Contents' in response:
                files = [obj['Key'] for obj in response['Contents'] if obj['Key'].endswith('.csv')]
        except Exception as e:
            print(f"Erro ao listar arquivos do bucket {self.bucket_name}: {e}")
        return files
    
    def download_files(self):
        """Baixa todos os arquivos CSV listados do S3 para o diretório local."""
        files = self.list_s3_files()
        for file_key in files:
            file_name = os.path.join(self.local_dir, os.path.basename(file_key))
            try:
                self.s3_client.download_file(self.bucket_name, file_key, file_name)
                print(f"Baixado: {file_key} -> {file_name}")
            except Exception as e:
                print(f"Erro ao baixar {file_key}: {e}")
    
    def concatenate_csv_files(self):
        """Lê e concatena os arquivos CSV baixados em um único DataFrame."""
        csv_files = [os.path.join(self.local_dir, f) for f in os.listdir(self.local_dir) if f.endswith('.csv')]
        df_list = []
        for file in csv_files:
            try:
                df = pd.read_csv(file)
                df_list.append(df)
            except Exception as e:
                print(f"Erro ao ler {file}: {e}")
        
        if df_list:
            return pd.concat(df_list, ignore_index=True)
        else:
            return pd.DataFrame()


    def download_from_s3(self):
        """
        Faz o download de um arquivo do S3.
        
        :param bucket_name: Nome do bucket no S3.
        :param object_key: Caminho/Chave do arquivo no S3.
        :param download_path: Caminho local onde o arquivo será salvo.
        """
      
        try:
            # Fazendo o download do arquivo
            self.s3_client.download_file(self.bucket_name, self.arq, self.new_arq)
            print(f"Download concluído: {self.local_dir}")
        except Exception as e:
            print(f"Erro ao fazer download do arquivo: {e}")

print("Done")

Done


In [8]:

bucket_name = "datathon-base"
prefix = "base_original/noticias/"
local_dir = '/kaggle/working/noticias/'
downloader = S3FileDownloader(bucket_name, prefix, local_dir)
downloader.download_files()
df_news = downloader.concatenate_csv_files()


Baixado: base_original/noticias/dados_2014_07.csv -> /kaggle/working/noticias/dados_2014_07.csv
Baixado: base_original/noticias/dados_2015_12.csv -> /kaggle/working/noticias/dados_2015_12.csv
Baixado: base_original/noticias/dados_2016_10.csv -> /kaggle/working/noticias/dados_2016_10.csv
Baixado: base_original/noticias/dados_2016_11.csv -> /kaggle/working/noticias/dados_2016_11.csv
Baixado: base_original/noticias/dados_2016_12.csv -> /kaggle/working/noticias/dados_2016_12.csv
Baixado: base_original/noticias/dados_2017_01.csv -> /kaggle/working/noticias/dados_2017_01.csv
Baixado: base_original/noticias/dados_2017_02.csv -> /kaggle/working/noticias/dados_2017_02.csv
Baixado: base_original/noticias/dados_2017_03.csv -> /kaggle/working/noticias/dados_2017_03.csv
Baixado: base_original/noticias/dados_2017_04.csv -> /kaggle/working/noticias/dados_2017_04.csv
Baixado: base_original/noticias/dados_2017_05.csv -> /kaggle/working/noticias/dados_2017_05.csv
Baixado: base_original/noticias/dados_20

In [9]:
df_news.head(3)

,page,url,issued,modified,title,body,caption
0,a5e207ec-4208-4e92-ac02-d37475f7000d,http://g1.globo.com/df/distrito-federal/notici...,2018-01-01 13:59:46+00:00,2018-01-01 13:59:47+00:00,Chuvas em 2017 ficam 15% abaixo do esperado pa...,Raios sobre região do DF\nFelipe Bastos/Arquiv...,Volume esperado para últimos doze meses era de...
1,afc82fd5-6c2a-4b78-8197-003dd7cb3757,http://g1.globo.com/df/distrito-federal/notici...,2018-01-08 13:08:43+00:00,2018-01-08 13:51:40+00:00,Brasília tem 79 apartamentos funcionais de R$ ...,Prédio com apartamentos funcionais no Plano Pi...,Imóveis ficam em região nobre da capital e são...
2,259593ec-c608-43fc-984d-9fc50b3ec1a9,http://g1.globo.com/ba/bahia/noticia/pai-e-pre...,2018-01-16 01:22:46+00:00,2018-01-16 01:25:55+00:00,Pai é preso após agredir filho de 6 anos com c...,Pai agride filho de 6 anos com golpes de vasso...,"Caso ocorreu na cidade de Feira de Santana, a ..."


In [10]:
bucket_name = "datathon-base"
prefix = "base_original/interacoes/"
local_dir = '/kaggle/working/interacoes/'
downloader = S3FileDownloader(bucket_name, prefix, local_dir)
downloader.download_files()
df_user = downloader.concatenate_csv_files()

Baixado: base_original/interacoes/dados_2022_07.csv -> /kaggle/working/interacoes/dados_2022_07.csv
Baixado: base_original/interacoes/dados_2022_08.csv -> /kaggle/working/interacoes/dados_2022_08.csv


In [11]:
df_user.head(3)

,userId,userType,historySize,history,timestampHistory,numberOfClicksHistory,timeOnPageHistory,scrollPercentageHistory,pageVisitsCountHistory,timestampHistory_new
0,e5f68d5e7cdbe56d6984589b4baa6ebfc5e8a8a918e57d...,Logged,12,7456c88d-6473-46ed-83a6-dfc406994162,2022-08-01 16:13:12.309,21,84995,43.48,1,2022-08-01 16:13:12.309
1,e5f68d5e7cdbe56d6984589b4baa6ebfc5e8a8a918e57d...,Logged,12,458bf0ec-efb4-4bfd-9446-c80295e6aa87,2022-08-05 11:03:37.494,79,536583,75.56,1,2022-08-05 11:03:37.494
2,e5f68d5e7cdbe56d6984589b4baa6ebfc5e8a8a918e57d...,Logged,12,89fa73f0-4341-4de4-bb2a-e429ef96bd43,2022-08-05 11:25:02.544,25,110000,45.06,1,2022-08-05 11:25:02.544


In [12]:
df_user.shape

(8123951, 10)

In [13]:
df_user.drop_duplicates(inplace=True)

In [14]:
df_news = df_news[['page','url','issued','title','caption','body']]
df_news.drop_duplicates(inplace=True)

In [15]:
padrao_colunas_user = {
    'userId': 'user_id',
    'history': 'news_id',
    'page': 'news_id',
}

df_user = df_user.rename(columns=padrao_colunas_user)
df_news = df_news.rename(columns=padrao_colunas_user)

print(df_user.columns)
print(df_news.columns)

# Exibir quantas interações e notícias restaram após o filtro
print(f"Notícias tamanho: {df_news.shape[0]}")
print(f"Interações tamanho: {df_user.shape[0]}")

Index(['user_id', 'userType', 'historySize', 'news_id', 'timestampHistory',
       'numberOfClicksHistory', 'timeOnPageHistory', 'scrollPercentageHistory',
       'pageVisitsCountHistory', 'timestampHistory_new'],
      dtype='object')
Index(['news_id', 'url', 'issued', 'title', 'caption', 'body'], dtype='object')
Notícias tamanho: 255603
Interações tamanho: 8123951


In [16]:
# Converter a coluna 'issued' (data da notícia) para datetime
df_news['issued'] = pd.to_datetime(df_news['issued'])

print(df_news['issued'].max())
print(df_news['issued'].min())

# Definir um limite de tempo (exemplo: últimos 30 dias)
data_limite = df_news['issued'].max() - pd.Timedelta(days=45)

# Filtrar notícias mais recentes
df_news = df_news[df_news['issued'] >= data_limite]

# Agora, filtramos df_user para manter apenas interações com notícias recentes
df_user = df_user[df_user['news_id'].isin(df_news['news_id'])]

# Exibir quantas interações e notícias restaram após o filtro
print(f"Notícias recentes: {df_news.shape[0]}")
print(f"Interações recentes: {df_user.shape[0]}")

2022-08-15 02:57:16+00:00
2014-07-31 08:00:01+00:00
Notícias recentes: 33941
Interações recentes: 6962404


In [17]:
df_news.shape

(33941, 6)

In [18]:
df_news.head(5)

,news_id,url,issued,title,caption,body
4566,2b5e38bf-10ac-4f83-be25-bc14ab441a3b,http://g1.globo.com/df/distrito-federal/notici...,2022-07-05 11:00:59+00:00,Homem filmado agredindo mulher no portão de ca...,Caso foi na última quarta-feira (29); vítima t...,Câmera de segurança gravou agressão de homem à...
4567,36a451f3-07a1-4c8c-99bf-6732a9cd6663,http://g1.globo.com/df/distrito-federal/notici...,2022-07-04 23:56:19+00:00,"Polícia investiga se acidente na Epia, no DF, ...",Outro carro foi atingido na noite de domingo (...,"Acidente na Epia Sul, no DF, no domingo (3) ap..."
4568,a396d8f4-c069-4ed6-b425-f2fccdb4443a,http://g1.globo.com/df/distrito-federal/notici...,2022-07-03 23:10:08+00:00,FOTOS: Parada LGBTQIA+ de Brasília realiza 23ª...,Concentração começou às 14h em frente ao Congr...,"Após dois anos suspensa, Parada LGBTQIA+ de Br..."
4569,e996de8d-b5d7-4824-9e44-35997faf4e2f,http://g1.globo.com/df/distrito-federal/notici...,2022-07-08 22:36:43+00:00,TCU identifica superfaturamento na compra de V...,"Ao todo, 15.120 comprimidos foram comprados po...",Viagra\nPixabay\nO Tribunal de Contas da União...
4570,810da14a-8be9-4c3d-8fb8-351997a6511a,http://g1.globo.com/df/distrito-federal/notici...,2022-07-08 19:08:18+00:00,"IPCA: inflação no DF fica em 0,81% em junho; m...",Dados do IBGE foram divulgados nesta sexta-fei...,Reajuste dos planos de saúde\nJN\nA inflação n...


In [19]:
df_news['news_content'] = df_news['title'] + ' ' + df_news['caption'] + ' ' + df_news['body']

In [20]:
vectorizer = TfidfVectorizer(stop_words=stopwords_ptbr)
tfidf_matrix = vectorizer.fit_transform(df_news['news_content'])

### Calcular a similaridade entre as notícias

In [21]:

cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)


In [22]:
# Função para recomendar notícias baseadas em conteúdo
def recommend_news_based_on_content(user_history, cosine_sim, df_news, num_recommendations=10):
    recommended_news = set()
    
    for news_id in user_history['news_id']:
        if news_id in news_id_to_index:  # Verifica se o ID está no mapeamento
            news_index = news_id_to_index[news_id]  # Obtém o índice correto
            
            # Obter os índices das notícias mais similares
            similar_indices = cosine_sim[news_index].argsort()[:-num_recommendations-1:-1]
            
            # Adicionar as notícias recomendadas
            recommended_news.update(df_news.iloc[similar_indices]['title'].values)
    
    return list(recommended_news)[:num_recommendations]

### Criar um dicionário para mapear news_id para índice na matriz cosine_sim

In [23]:
news_id_to_index = {news_id: idx for idx, news_id in enumerate(df_news['news_id'])}

### Exemplo de histórico do usuário

In [24]:
user = '1505326617b9465f6e13eb1d0d9782bff2af61822a7bc780fa058e95851d15ee'
user_history = df_user[df_user['user_id'] == user]
user_history = user_history[['user_id', 'news_id']]
user_history = user_history[user_history['news_id'].isin(df_news['news_id'])]

In [25]:
# Recomendando notícias com base no conteúdo do histórico
recommended_news = recommend_news_based_on_content(user_history, cosine_sim, df_news, num_recommendations=5)
print(recommended_news)

['Infográfico mostra como foi movimentação no ataque a Shinzo Abe', 'Ex-premiê Shinzo Abe morre após ser baleado no Japão', 'Vídeos mostram diferentes ângulos do ataque ao ex-premiê do Japão, Shinzo Abe; VEJA', 'Novo vídeo mostra que Shinzo Abe foi atingido pelas costas; VEJA']


## AWS - S3 - Configuração para subir o modelo treinado

In [26]:
import os
import boto3
import pickle
from io import BytesIO
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()

# Configurações da AWS
AWS_ACCESS_KEY = user_secrets.get_secret("AWS_ACCESS_KEY_ID")
AWS_SECRET_KEY = user_secrets.get_secret("AWS_SECRET_ACCESS_KEY")
AWS_SESSION_TOKEN = user_secrets.get_secret("AWS_SESSION_TOKEN")
AWS_REGION = user_secrets.get_secret("AWS_REGION")
BUCKET_NAME = "datathon-base"

# Criar conexão com o S3
def get_s3_client():
    return boto3.client(
        "s3",
        aws_access_key_id=AWS_ACCESS_KEY,
        aws_secret_access_key=AWS_SECRET_KEY,
        aws_session_token=AWS_SESSION_TOKEN,
        region_name=AWS_REGION,
    )

# Função para salvar o modelo no S3
def save_model_to_s3(model_data, s3_path="models/news_recommendation_model_history.pkl"):
    s3 = get_s3_client()
    
    # Serializar o modelo
    with open("news_recommendation_model_history.pkl", "wb") as f:
        pickle.dump(model_data, f)

    # Fazer upload para o S3
    with open("news_recommendation_model_history.pkl", "rb") as f:
        s3.upload_fileobj(f, BUCKET_NAME, s3_path)

    print(f"Modelo salvo no S3 em: s3://{BUCKET_NAME}/{s3_path}")

# Função para carregar o modelo do S3
def load_model_from_s3(s3_path="models/news_recommendation_model_history.pkl"):
    s3 = get_s3_client()
    
    # Baixar o arquivo do S3
    with open("news_recommendation_model_history.pkl", "wb") as f:
        s3.download_fileobj(BUCKET_NAME, s3_path, f)

    # Carregar o modelo
    with open("news_recommendation_model_history.pkl", "rb") as f:
        model_data = pickle.load(f)

    print("Modelo carregado com sucesso!")
    return model_data



### Salvando o modelo na AWS

In [27]:
save_model_to_s3(cosine_sim)

Modelo salvo no S3 em: s3://datathon-base/models/news_recommendation_model_history.pkl
